# CMPE 493 Assignment 2

In this project, we were supposed to implement a movie recommendation system using vector model and tf-idf weights as dimensions.

In the project, storylines are treated as documents and weights are constructed using storylines.

To sum up the project, first the html datas are scraped from imdb and title, storyline and recommended movie ids are extracted from html response without using HtmlParser. At this stage, html characters, stopwords and punctuations are removed to incease accuracy.

Then, frequencies are found and tf-idf weights are constructed to complete the document vectors.

In recommendations part, similarities among different movies are checked using cosine similarity and most similar ones are recommended.

Finally, in the evaluations part, results are evaluated with recall, precision and f1-scores

As a bonus, I have also implemented a clustering system.

Necessary imports

In [2]:
import requests
from collections import Counter
import math
import collections
import operator

In [3]:
ids = []          # list to hold imdb ids

url = "https://www.imdb.com/title/"    # imdb url

main_dict = dict()   # dict to have imdb id as key and list consisting of title, storyline and recommended movies as value

document_number = 1199   # total number of imdb ids and thus total number of storylines = documents
term_frequencies = dict()    # dict that has imdb_id as key and term freq of all the terms in its storyline as value
document_frequencies = dict()   # dict that has term as key and number of document that has that word as value
weights = dict()   # dict that has imdb id as key and tf-idf weights of its storyline as value


To have better results and more sound storylines and titles, I have removed html characters as well as punctuations and stopwords.

In [4]:
# Replace html characters from the corpus
def replace_html_characters(text):
    text = text.replace("\\n", " ")
    text = text.replace("&lt;", " ")
    text = text.replace("&#5;", " ")
    text = text.replace("&#22;", " ")
    text = text.replace("&#127;", " ")
    text = text.replace("&#1;", " ")
    text = text.replace("&amp;", " ")
    text = text.replace("&#30;", " ")
    text = text.replace("&#2;", " ")
    text = text.replace("&#27;", " ")
    text = text.replace("&#3;", " ")
    text = text.replace("&#31;", " ")
    text = text.replace("&nbsp", " ")
    return text

In [5]:
# Remove punctuations from the text
def remove_punctuations(text):
    punctuations = ['"', '!', '^', '%', '<', '+', '~', '*', ';', ':', '(', '?', '&', '}', ']', '|', ',', "'", ')', '-', '#', '`', '@', '/', '$', '_', '{', '.', '>', '[', '\\', '=']
    for punctuation in punctuations:
        text = text.replace(punctuation, " ")
    return text


In [6]:
# remove stopwords from the text
def remove_stopwords(text):
    stopwords = ['a', 'all', 'an', 'and', 'any', 'are', 'as', 'be', 'been', 'but', 'by ', 'few', 'for', 'have', 'he', 'her', 'here', 'him', 'his', 'how', 'i', 'in', 'is', 'it', 'its', 'many', 'me', 'my', 'none', 'of', 'on ', 'or', 'our', 'she', 'some', 'the', 'their', 'them', 'there', 'they', 'that ', 'this', 'us', 'was', 'what', 'when', 'where', 'which', 'who', 'why', 'will', 'with', 'you', 'your']
    text = " ".join(filter(lambda word: word not in stopwords, text.split()))
    return text


## IMDB Scraping

IMDB Scraping is pretty straightforward although the code looks complicated: get_movie_contents(imdb_id) makes a get request to imdb url with the given movie imdb id, then for each line in the html response, it makes the necessary parsing operations: 

* It searches for title_wrapper div to extract title of the movie
* It searches for storyline h2 header to find the start of the storyline and keeps axtracting the storyline until it finds a span closing tag 
* It searches for rec_item div to extract the recommended movies section

With the data it has extracted, it adds to the main_dict which has imdb_id of the movie as key and list of title, storyline and recomended movies, in that order.

Finally get_movie_contents(imdb_id) method returns title, storyline and recommended movies for the given imdb id, in that order.

In [7]:
def get_movie_contents(imdb_id):
    global main_dict
    if imdb_id in main_dict.keys():
        return main_dict[imdb_id][0], main_dict[imdb_id][1], main_dict[imdb_id][2]

    try:
        title = ""
        title_found = False

        storyline = ""
        storyline_title_found = False
        storyline_found = False

        recs = []
        r = requests.get(url + imdb_id)

        #r = r.strip()
        #print(r)
        for line in r.iter_lines():
            line = line.decode()
            #line = line.strip()
            #print(line)
            if title_found and line:
                title = replace_html_characters(line[line.index(">") + 1:line.index(";")])
                title_found = False
            if storyline_title_found and "<span>" in line:
                storyline_found = True
            if storyline_found:
                if "</span>" in line:
                    storyline_found = False
                    storyline_title_found = False
                storyline = storyline + " " + line
                storyline = storyline.replace("<span>", " ")
                storyline = storyline.replace("</span>", " ")
                storyline = remove_stopwords(replace_html_characters(remove_punctuations(storyline.lower())))
            if "<div class=\"title_wrapper\">" in line:
                title_found = True
            if "<h2>Storyline</h2>" in line:
                storyline_title_found = True
            if "<div class=\"rec_item\"" in line:
                index1 = line.rfind("\"")
                index2 = line[:index1].rfind("\"")
                rec = line[index2 + 1:index1]
                recs.append(rec)

        main_dict[imdb_id] = [title, storyline, recs]
        
        print(imdb_id)
        print(main_dict[imdb_id])
        print()
        
        return main_dict[imdb_id][0], main_dict[imdb_id][1], main_dict[imdb_id][2]

    except Exception as e:
        print(e)

In [8]:
# get all the movie ids from the file
with open("./movie_ids.csv", "r") as ids_file:
    global ids
    for line in ids_file:
        ids.append(line[:-1])

In [8]:
# make imdb scraping for all the movies in the movie ids file
global ids
for imdb_id in ids:
    #print(ids.index(imdb_id))
    print(get_movie_contents(imdb_id))

tt0112851
['Desperado ', 'director robert rodriguez picks up successful independent debut el mariachi left off slam bang south border action saga bucho joaquim dealmeida wealthy casually bloodthirsty drug kingpin rules seedy mexican border town bucho men make mistake angering el mariachi antonio banderas former musician now carries arsenal guitar case bucho responsible death el mariachi s girlfriend put bullet through fretting hand making unable to play guitar bent on revenge musician turned killing machine arrives town to put bucho out business though finds allies except carolina salma hayek runs bookstore that doesn t seem to attract readers desperado features supporting performances from cheech marin cynical bartender steve buscemi cantina patron sets up story quentin tarantino man really terrible joke to tell', ['tt0285823', 'tt0104815', 'tt0116367', 'tt0985694', 'tt1077258', 'tt1028528', 'tt0113101', 'tt0119396', 'tt0401792', 'tt0110632', 'tt0462322', 'tt0120746']]

('Desperado ',

tt1638002
['Love, Rosie ', 'from age 5 rosie dunne alex stewart best friends take on life just end up getting separated time time again comes to love s just everyone else each other realise meant to together', ['tt2674426', 'tt2582846', 'tt0989757', 'tt1666801', 'tt0281358', 'tt1355630', 'tt3846674', 'tt0332280', 'tt1606389', 'tt3799232', 'tt2726560', 'tt1702439']]

('Love, Rosie ', 'from age 5 rosie dunne alex stewart best friends take on life just end up getting separated time time again comes to love s just everyone else each other realise meant to together', ['tt2674426', 'tt2582846', 'tt0989757', 'tt1666801', 'tt0281358', 'tt1355630', 'tt3846674', 'tt0332280', 'tt1606389', 'tt3799232', 'tt2726560', 'tt1702439'])
tt0384814
['Ask the Dust ', 'l early 1930 s racism poverty disease color bunker hill neighborhood arturo bandini lover men beasts alike has arrived from colorado to write great los angeles novel after six months down to last nickel orders cup coffee served by camilla lopez

tt0158622
['The Flintstones in Viva Rock Vegas ', 'flintstones at again flintstones rubbles head rock vegas fred hoping to court lovely wilma nothing stand way love except conniving chip rockefeller playboy born baysville has made cutthroat town rock vegas fred win wilma s love', ['tt0109813', 'tt0141369', 'tt0301454', 'tt0119310', 'tt0322389', 'tt0104437', 'tt0119190', 'tt0053502', 'tt0424774']]

('The Flintstones in Viva Rock Vegas ', 'flintstones at again flintstones rubbles head rock vegas fred hoping to court lovely wilma nothing stand way love except conniving chip rockefeller playboy born baysville has made cutthroat town rock vegas fred win wilma s love', ['tt0109813', 'tt0141369', 'tt0301454', 'tt0119310', 'tt0322389', 'tt0104437', 'tt0119190', 'tt0053502', 'tt0424774'])
tt0048641
['Sommarnattens leende ', 's late nineteenth century sweden middle aged lawyer fredrik egerman nineteen year old current wife anne egerman s two year marriage has not yet consummated fredrik wants to

tt0050710
['Mister Cory ', 'cory ambitious chicago slum kid knack gambling gets busboy job at posh wisconsin resort real purpose to gamble staff guests romance rich young ladies setbacks follow cory eventually rises to high position world professional gambling just can t forget glamorous vollard sisters now has even farther to fall', ['tt0052059', 'tt0050704', 'tt0048628', 'tt0053879', 'tt0045175', 'tt0049663', 'tt0048654', 'tt0045780', 'tt0043935', 'tt0064688', 'tt0054473', 'tt0045490']]

('Mister Cory ', 'cory ambitious chicago slum kid knack gambling gets busboy job at posh wisconsin resort real purpose to gamble staff guests romance rich young ladies setbacks follow cory eventually rises to high position world professional gambling just can t forget glamorous vollard sisters now has even farther to fall', ['tt0052059', 'tt0050704', 'tt0048628', 'tt0053879', 'tt0045175', 'tt0049663', 'tt0048654', 'tt0045780', 'tt0043935', 'tt0064688', 'tt0054473', 'tt0045490'])
tt2488496
['Star Wars

tt0800320
['Clash of the Titans ', 'perseus son king gods zeus caught war between gods mortals family killed nothing left to lose perseus gathers war band to help conquer kraken medusa hades god underworld', ['tt1646987', 'tt0473075', 'tt1253864', 'tt1428538', 'tt0401729', 'tt0489099', 'tt1351685', 'tt1464540', 'tt1386703', 'tt1104001', 'tt0409847', 'tt0829150']]

('Clash of the Titans ', 'perseus son king gods zeus caught war between gods mortals family killed nothing left to lose perseus gathers war band to help conquer kraken medusa hades god underworld', ['tt1646987', 'tt0473075', 'tt1253864', 'tt1428538', 'tt0401729', 'tt0489099', 'tt1351685', 'tt1464540', 'tt1386703', 'tt1104001', 'tt0409847', 'tt0829150'])
tt0119217
['Good Will Hunting ', 'touching tale wayward young man struggles to find identity living world can solve problem except one brewing deep within himself until one day meets soul mate opens mind heart', ['tt0268978', 'tt0097165', 'tt0169547', 'tt0120382', 'tt0109830',

...

tt0105402
['Sidekicks ', 'barry asthmatic kid having trouble life lives father computer programmer texas barry struggling to get by life dealing rough school life bullies well health barry s only source enjoyment fantasizing that chuck norris barry becomes sick getting picked on by bigger guys decides to learn karate hopes one day meeting great chuck norris', ['tt0104670', 'tt0102045', 'tt0107101', 'tt0094792', 'tt0099399', 'tt0089604', 'tt0114697', 'tt0091055', 'tt0087727', 'tt0176943', 'tt0083960', 'tt0095296']]

('Sidekicks ', 'barry asthmatic kid having trouble life lives father computer programmer texas barry struggling to get by life dealing rough school life bullies well health barry s only source enjoyment fantasizing that chuck norris barry becomes sick getting picked on by bigger guys decides to learn karate hopes one day meeting great chuck norris', ['tt0104670', 'tt0102045', 'tt0107101', 'tt0094792', 'tt0099399', 'tt0089604', 'tt0114697', 'tt0091055', 'tt0087727', 'tt017694

tt0119396
['Jackie Brown ', 'middle aged stewardess jackie brown smuggles money from mexico to los angeles arms dealer ordell robbie gets caught by agents ray nicolet mark dargus ten thousand dollars cocaine purse propose deal to to help to arrest ordell exchange freedom meanwhile ordell asks 56 year old max cherry runs bail bond business to release jackie brown intention eliminating jackie suspects ordell s intention plots complicated confidence game max to steal half million dollars from ordell', ['tt1028528', 'tt0378194', 'tt3460252', 'tt0116367', 'tt0266697', 'tt0105236', 'tt0401792', 'tt0361748', 'tt0113101', 'tt1077258', 'tt1853728', 'tt0110632']]

('Jackie Brown ', 'middle aged stewardess jackie brown smuggles money from mexico to los angeles arms dealer ordell robbie gets caught by agents ray nicolet mark dargus ten thousand dollars cocaine purse propose deal to to help to arrest ordell exchange freedom meanwhile ordell asks 56 year old max cherry runs bail bond business to rel

tt0014624
['A Woman of Paris: A Drama of Fate ', 'marie st clair believes has jilted by artist fiance jean fails to meet at railway station goes off to paris alone year later mistress wealthy pierre revel meets jean again misinterpreting events bounces back forth between apparent security true love', ['tt0014358', 'tt0050598', 'tt0039631', 'tt0018773', 'tt0009611', 'tt0013486', 'tt0012304', 'tt0044837', 'tt0009018', 'tt0010057', 'tt0010747', 'tt0050243']]

('A Woman of Paris: A Drama of Fate ', 'marie st clair believes has jilted by artist fiance jean fails to meet at railway station goes off to paris alone year later mistress wealthy pierre revel meets jean again misinterpreting events bounces back forth between apparent security true love', ['tt0014358', 'tt0050598', 'tt0039631', 'tt0018773', 'tt0009611', 'tt0013486', 'tt0012304', 'tt0044837', 'tt0009018', 'tt0010057', 'tt0010747', 'tt0050243'])
tt0107331
['De kleine blonde dood ', 'poet valentijn boecke meets former teacher mieke sh

tt0054777
['The Curse of the Werewolf ', 'eighteenth century spain beggar comes to castle cruel marquee on wedding day to beg food marque locks dungeon forgotten mute daughter gaoler feeds along years grows up widower marquee unsuccessfully tries to shag locks servant dungeons beggar that rapes released kills marquee flees to forest found living like animal woods by don alfredo brings home soon servant teresa finds that pregnant gives birth to boy on christmas dies boy leon raised by don alfredo teresa years later learn curse that boy carries local priest advises that must raised love happen to leon', ['tt0053677', 'tt0056277', 'tt0057226', 'tt0058073', 'tt0056347', 'tt0057401', 'tt0053085', 'tt0050894', 'tt0050280', 'tt0058405', 'tt0059127', 'tt0060841']]

('The Curse of the Werewolf ', 'eighteenth century spain beggar comes to castle cruel marquee on wedding day to beg food marque locks dungeon forgotten mute daughter gaoler feeds along years grows up widower marquee unsuccessfully t

tt0010747
['Sunnyside ', 'charlie works on farm from 4am to late at night gets food on run milking cow into coffee holding chicken over frying pan to get fried eggs loves neighbor s daughter edna disliked by father rides cow into stream kicked off unconscious dreams nymph dance back reality city slicker hurt car crash being cared by edna charlie rejected after attempting to imitate slicker result ambiguous either tragic happy ending critics long argued to whether final scene real dream', ['tt0010057', 'tt0012304', 'tt0013486', 'tt0009611', 'tt0014358', 'tt0009018', 'tt0007832', 'tt0004936', 'tt0007264', 'tt0006684', 'tt0006689', 'tt0007194']]

('Sunnyside ', 'charlie works on farm from 4am to late at night gets food on run milking cow into coffee holding chicken over frying pan to get fried eggs loves neighbor s daughter edna disliked by father rides cow into stream kicked off unconscious dreams nymph dance back reality city slicker hurt car crash being cared by edna charlie rejected a

## Tf-idf model

To construct the tf-idf model, first document and term frequencies are gathered, then the weights are constructed.

dictionary __term_frequencies__ has imdb_id as key and a list of (term, frequency) pairs for that movie's storyline as value.

dictionary __document_frequencies__ has term as key and number of documents that term appears in as value.

dictionary __weights__ has imdb_id as key and a dictionary that has terms as keys and their weights as values as value. In other words its key is imdb_id and value is another dictionary.

In [9]:
def get_frequencies():
    global main_dict
    global document_frequencies
    global term_frequencies
    for key in main_dict.keys():
        term_frequencies[key] = Counter(main_dict[key][1].split()).most_common() # update term frequencies of document
        for word in set(main_dict[key][1].split()):    # update the document frequencies of the terms
            document_frequencies.setdefault(word, 0)
            document_frequencies[word] += 1 
        
def get_weights():
    global main_dict
    global weights
    for key in main_dict.keys():
        weights.setdefault(key, [])
        document_weights = dict()   # temporary dict to hold weights of one document
        for word in set(main_dict[key][1].split()):   # for each term in the movie's storyline
            term_frequency = 0
            for pair in term_frequencies[key]:   # get its term frequency from the term_frequencies dictionary first
                if pair[0] == word:
                    term_frequency = pair[1]    
                    break
            document_weights.setdefault(word, 0)   # then calculate its tf-idf weight
            document_weights[word] = (1 + math.log(term_frequency)) * math.log(document_number / document_frequencies[word])
        document_weights = list(reversed(sorted(document_weights.items(), key=operator.itemgetter(1))))   # sort the weights
        document_weights = dict(document_weights)
        weights[key] = document_weights
        

In [10]:
get_frequencies()
get_weights()


## Recommendation

Recommendation is done with finding all the consine similarity values for a given value, sorting the movies according to their similarity values and returning first K of them.

In the __recommend(movie, number_of_recommendation=12)__ method, dictionary __similarities__ holds movie ids as key and their similarity values regarding the given imdb id in the parameter __movie__ as values. The method first construct that dictionary by iterating through all the movies (except the movie given as the parameter obviously) and constructs a list by sorting the __similarities__ dict by their values and getting the first __number_of_recommendation__ items of it.

In __recommend(imdb_id)__, there was no __number_of_recommendation__ parameter, so I assumed it will be default 12.

__get_similarity(id1, id2)__ returns cosine similarity of the 2 given movies.

__print_recommendations__ just prints all the recommendations for all the movies in the __ids__ list.

In [11]:
def recommend(imdb_id):
    return recommend(imdb_id, 12)

In [12]:
# recommend number_of_recommendation number of movies for the given movie
def recommend(movie, number_of_recommendation=12):
    similarities = dict()
    similarity_list = []
    for movie_id in ids:   # construct the dictionary
        if movie == movie_id:
            continue
        else:
            similarities[movie_id] = get_similarity(movie, movie_id)
    for item in list(reversed(sorted(similarities.items(), key=operator.itemgetter(1))))[:number_of_recommendation]:
        similarity_list.append(item[0])
    return similarity_list

In [13]:
# get cosine similarity for the 2 given movie ids.
def get_similarity(id1, id2):
    global weights
    weights1 = weights[id1]
    weights2 = weights[id2]
    numerator = sum(weights1[word] * weights2[word] for word in list(set(list(weights1.keys())[:]) & set(list(weights2.keys())[:])))
    denominator = math.sqrt(sum(i * i for i in list(weights1.values())[:])) * math.sqrt(sum(i * i for i in list(weights2.values())[:]))
    try:
        return numerator / denominator
    except ZeroDivisionError:
        return 0
    

In [14]:
def print_recommendations():
    global ids
    global main_dict
    for id in ids:
        print(data[id][0])
        for movie_id in recommend(id, 12):
            print(main_dict[movie_id[0]][0])
        print("****************") 

## Evaluation

In the evaluation part, what the methods does is pretty straightforward with their names.

While calculating evaluation scores, relevant documents are considered as the recommended movies by the imdb, regardless of the number of recommendations done by the project itself. For example, if it has been asked to make only 1 recommendation, recall will be 1/12 maximum assuming imdb recommended 12 movies for that movie. In other words, relevant documents are always the ones recommended by the imdb.

In __get_precision_from_lists__ and __get_precision__, precision is calculated as the number of movies that are recommended by both the project and imdb, over the number of recommendations by the project.

In __get_recall_from_lists__ and __get_recall__, recall is calculated as the number of movies that are recommended by both the project and imdb, over the number of recommendations by imdb which is 12 most of the time.

f1 score is 2 x recall x precision / (precision + recall)

In [15]:
def get_precision_from_lists(rec_movie_ids, relevant_movie_ids, K):
    return (len(set(rec_movie_ids) & set(relevant_movie_ids))) / K * 1.0

In [16]:
def get_recall_from_lists(rec_movie_ids, relevant_movie_ids, K):
    return (len(set(rec_movie_ids) & set(relevant_movie_ids))) / (len(relevant_movie_ids) * 1.0)

In [17]:
def get_f1_score_from_lists(rec_movie_ids, relevant_movie_ids, K):
    if get_precision_from_lists(rec_movie_ids, relevant_movie_ids, K) == 0 or get_recall_from_lists(rec_movie_ids, relevant_movie_ids, K) == 0:
        return 0
    else:
        return (2 * get_precision_from_lists(rec_movie_ids, relevant_movie_ids, K) * get_recall_from_lists(rec_movie_ids, relevant_movie_ids, K) / (get_precision_from_lists(rec_movie_ids, relevant_movie_ids, K) + get_recall_from_lists(rec_movie_ids, relevant_movie_ids, K)))
    

In [18]:
def get_precision(movie_id, number_of_recommendation):
    global main_dict    
    return (len(set(recommend(movie_id, number_of_recommendation)) & set(main_dict[movie_id][2]))) / (number_of_recommendation * 1.0)


In [19]:
def get_recall(movie_id, number_of_recommendation):
    global main_dict
    return (len(set(recommend(movie_id, number_of_recommendation)) & set(main_dict[movie_id][2]))) / (len(main_dict[movie_id][2]) * 1.0)


In [20]:
def get_f1_score(movie_id, number_of_recommendation):
    if get_precision(movie_id, number_of_recommendation) == 0 or get_recall(movie_id, number_of_recommendation) == 0:
        return 0
    else:
        return (2 * get_precision(movie_id, number_of_recommendation) * get_recall(movie_id, number_of_recommendation)) / (get_precision(movie_id, number_of_recommendation) + get_recall(movie_id, number_of_recommendation))
    

In [21]:
def evaluate_recommendations(rec_movie_ids, relevant_movie_ids, K):
    return get_precision_from_lists(rec_movie_ids, relevant_movie_ids, K), get_recall_from_lists(rec_movie_ids, relevant_movie_ids, K), get_f1_score_from_lists(rec_movie_ids, relevant_movie_ids, K)
    """
    Gets list of recommended and relevant movie ids and K value.
    
    Returns precision, recall, F1 values for K respectively. 
    """


In [ ]:
print(evaluate_recommendations(recommend("tt2267968"), main_dict["tt2267968"][2], 12))

## Bonus: Clustering

As a bonus, I have also implemented a bottom-up clustering algorithm that would start eacy movie as a cluster and merge them one by one with the most similar ones first, until the number of clusters is reached or similarity limit is observed.

In [23]:
# cluster class
class Cluster:
    def __init__(self, imdb_id, weights, number_of_elements):
        self.weights = weights
        self.number_of_elements = number_of_elements
        self.imdb_ids = imdb_id

In merging clusters, their weights will be updated, in other words, its centroid will be constructed with taking the weighted average of two merged clusters' weights, with the number of elements in those clusters being the weights. 
__merge_clusters(cluster1, cluster2)__ takes 2 clusters as parameter and returns the resulting cluster of their merge.

In [24]:
# merge clusters
def merge_clusters(cluster1, cluster2):
    new_number_of_elements = cluster1.number_of_elements + cluster2.number_of_elements
    new_weights = dict()
    new_ids = list(set().union(cluster1.imdb_ids, set(cluster2.imdb_ids)))
    for word in list(set().union(cluster1.weights.keys(), set(cluster2.weights.keys()))): # form new weights
        cluster1.weights.setdefault(word, 0)
        cluster2.weights.setdefault(word, 0)
        new_weights[word] = (cluster1.weights[word] * cluster1.number_of_elements + cluster2.weights[word] * cluster2.number_of_elements) / new_number_of_elements
    return Cluster(new_ids, new_weights, new_number_of_elements)

__get_cluster_similarity(cluster1, cluster2)__ is almost the same as __get_similarity__ method, it rather takes 2 clusters as argument and returns their cosine similarity.

In [25]:
def get_cluster_similarity(cluster1, cluster2):
    weights1 = cluster1.weights
    weights2 = cluster2.weights
    numerator = sum(weights1[word] * weights2[word] for word in list(set(list(weights1.keys())[:]) & set(list(weights2.keys())[:])))
    denominator = math.sqrt(sum(i * i for i in list(weights1.values())[:])) * math.sqrt(sum(i * i for i in list(weights2.values())[:]))
    try:
        return numerator / denominator
    except ZeroDivisionError:
        return 0

__find_closest_clusters(clusters)__ gets a list of clusters and returns the 2 most similar ones, the ones with the highest similarity score, or lowest distance between their centroids if distance is defined as 1 - cosine-similarity. Its complexity is n^2 for n being the length of __clusters__ length. 

In [26]:
def find_closest_clusters(clusters):
    similarity_score = 0
    closest_clusters = []
    for cluster1 in clusters:
        for cluster2 in clusters:
            if cluster1 == cluster2:
                continue
            if get_cluster_similarity(cluster1, cluster2) > similarity_score:
                similarity_score = get_cluster_similarity(cluster1, cluster2)
                closest_clusters = [cluster1, cluster2]
    return closest_clusters

__find_clusters(clusters, number_of_final_clusters=0, similarity_limit=0)__ takes 3 arguments:
__clusters__: clusters list
__number_of_final_clusters__: number of clusters after the merging is complete
__similarity_limit__: similarity limit while merging clusters. All the clusters that have a higher similarity than that limit will be merged.

This method checks 2 most similar clusters in the __clusters__ list, merges them, removes the old merged clusters from the list and adds newly formed cluster which is the result of merge operation.

The best way to use it is to give a __similarity_limit__ and not bother with __number_of_final_clusters__, as it would be an arbitrary number if you don't have a lot of knowledge about the dataset of ids.

When tested with __similarity_limit__ of 0.30, it merged all the movies about the exact same topic or movies of continuation. Firstly King Kong movies of different years were merged, then different Family Guy movies, WrestleMania series, followed by Garfield, The Hunchback of Notre Dame and Shrek movies etc. In other words, it is a great way to group not similar movies but same movies or movie series. Then another recommendation algorithm might be run on the clusters, which would check similarity of the movies with the cluster that the given movie is in, i.e. when recommending for Dracula, instead of checking all the similarities for all the movies with the given Dracula movie, we could check the similarities for all the movies with the Dracula cluster, which has at least 5 movies ( cluster consisting of  'tt0021815', 'tt0079073', 'tt0021814' was merged with cluster consisting of 'tt0079641', 'tt0051554' with a similarity score of 0.3336280581141756.) This method could give more reliable results and also provide us a more trustable vector model, as normally we take only one storyline to construct the vector and it can be very short sometimes, but when the clustering is used, average of 5 different storylines of the basically same movie would be used. __recommend_by_clusters(movie, number_of_recommendation=12)__ just does that, but with movie id size of 1199, it takes around 20 mins for clusters to converge with __similarity_limit__ of 0.30 because of the n^2 complexity and it might be an overkil to use that algorithm for recommendation while norma recommendation system also does fine most of the time. 



In [32]:
def find_clusters(clusters, number_of_final_clusters=0, similarity_limit=0):
    global main_dict
    if similarity_limit == 0 and number_of_final_clusters == 0:
        return
    while len(clusters) > number_of_final_clusters:
        closest_clusters = find_closest_clusters(clusters)
        similarity_score = get_cluster_similarity(closest_clusters[0], closest_clusters[1])
        if similarity_score >= similarity_limit:
            print(closest_clusters[0].imdb_ids)
            print(closest_clusters[1].imdb_ids)
            print(similarity_score)
            new_cluster = merge_clusters(closest_clusters[0], closest_clusters[1])
            clusters.remove(closest_clusters[0])
            clusters.remove(closest_clusters[1])
            clusters.append(new_cluster)
        else:
            break
    return clusters

In [33]:
# recomend using clustering
# takes movie as imdb_id and number_of_recommendation 
def recommend_by_clusters(movie, clusters, number_of_recommendation=12):
    similarities = dict()
    similarity_list = []
    for cluster in clusters:   # find the cluster the movie belongs
        if movie in cluster.imdb_ids:
            own_cluster = cluster
            break
    for movie_id in ids:   # construct the dictionary
        if movie == movie_id:
            continue
        else:
            similarities[movie_id] = get_cluster_similarity(own_cluster, Cluster(movie_id, weights[movie_id], 1))
    for item in list(reversed(sorted(similarities.items(), key=operator.itemgetter(1))))[:number_of_recommendation]:
        similarity_list.append(item[0])
    return similarity_list

In [ ]:
clusters = [Cluster([key], weights[key], 1) for key in ids]
new_clusters = find_clusters(clusters, similarity_limit=0.30)

print(recommend_by_clusters("tt0356634", new_clusters, 12))